In [156]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')


import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
"""
PREP DATA
"""

from get_data import load_history, load_current, combine_history, process_y, load_bios, load_team_data
from model_training import make_predictions

# load_history()
# load_current()
# combine_history()
# load_team_data()
# load_bios()
# process_y()
# make_predictions()

import yahoo_utils
# yahoo_utils.refresh_players()
# yahoo_utils.refresh_player_games()
# yahoo_utils.refresh_teams()

In [466]:
from model_training import load_predictions
import lineup_utils
import yahoo_utils

        
class TeamAnalytics():
    
    def __init__(self, mode='season', date=None, week=1, team_key=None):
        self.mode = mode
        
        if mode == 'season':
            self.date = pd.Timestamp.now(tz='America/Los_Angeles').date()
            self.matchup = None
        
        elif mode == 'week':
            self.matchup = yahoo_utils.get_matchup(week=week)
            self.date = self.matchup['date_start'] - pd.Timedelta('1d')
            
        else:
            raise ValueError('Invalid value for mode')

        self.players = yahoo_utils.load_players()
        self.preds, self.player_stds = load_predictions()
        self.player_games = yahoo_utils.load_player_games()
        
        self.team_key = team_key
        if self.team_key is not None:
            self.current_team = self.players[self.players.team_key == team_key].index
            self.preds = self.preds.loc[self.current_team]
            self.player_games = self.player_games.loc[self.current_team]
        
        # could be populated instead of hardcoded
        self.cats = ['g','a','sog','fow','hit','block','pim','plusmin','ppp', 'ga','win','so','save']
        
        
        self.selected_team = []        
        self.n_games_lineup = 0
        self.own_totals = pd.Series({c:0 for c in self.cats})
        
        self.rest_games = None
        self.lineup_preds = None
        self.rel_lineup_preds = None
        self.get_lineup_preds()
    
    def get_lineup_preds(self):
        self.rest_games = lineup_utils.get_rest_of_period_games(self.date, self.player_games.reset_index(), self.selected_team, self.players, self.preds)
        stats_available = rest_games[rest_games.index.isin(self.preds.index) & ~rest_games.index.isin(self.selected_team)].index
        self.lineup_preds = self.preds.loc[stats_available, self.cats].apply(lambda x: x * self.rest_games[stats_available])
        self.lineup_preds['ga'] = self.preds.loc[stats_available, 'ga']
        self.lineup_preds['games_left'] = rest_games.loc[stats_available]
        
        if self.mode == 'week':
            comparison = [p for p in self.players.loc[self.players.opp_lineup].index if p in self.lineup_preds.index]
        else:
            comparison = [p for p in self.players.loc[self.players.is_rostered].index if p in self.lineup_preds.index]
      

        # standard deviation that takes into account individual players' uncertainty
        std_factor =  ( (self.preds.loc[comparison, self.cats].var()) + (self.player_stds / (17**0.5))**2 ) ** 0.5
        lineup_len_divisors = (self.lineup_preds.notna() + self.preds.loc[self.selected_team, self.cats].count()).apply(lambda x: x * (self.n_games_lineup + self.rest_games.mean()) / (len(self.selected_team)+1))

        self.rel_lineup_preds = ((self.lineup_preds.fillna(0) + self.own_totals)/lineup_len_divisors - self.preds.loc[comparison, self.cats].mean()) / std_factor
        self.lineup_preds['rank'] = self.rel_lineup_preds.sum(axis=1)
        self.lineup_preds['games_left'] = self.rest_games.loc[stats_available]
        
        return self
    
    
    def select_player(self, selected_player):
        
        self.n_games_lineup += self.rest_games.loc[selected_player]
        self.selected_team.append(selected_player)
        self.own_totals += self.lineup_preds.loc[selected_player, self.cats].fillna(0)
        self.own_totals['ga'] = self.preds.loc[self.selected_team, 'ga'].mean()
        
        self.get_lineup_preds()
        return self
    
    def unselect_player(self, selected_player):
        self.selected_team.remove(selected_player)
        self.get_lineup_preds()
        
        self.own_totals -= self.lineup_preds.loc[selected_player, self.cats].fillna(0)
        self.own_totals['ga'] = self.preds.loc[self.selected_team, 'ga'].mean()
        
        self.n_games_lineup -= self.rest_games.loc[selected_player]
        return self
    
    def render(self):
        return self.players.join(self.lineup_preds, how='inner').sort_values('rank')
    
    def make_full_selections(self):
        if self.team_key is None:
            team_max_length = 17
        else:
            team_max_length = len(self.current_team)
        
        while len(self.selected_team) < team_max_length and len(self.lineup_preds) > 0:
            selection = self.lineup_preds['rank'].idxmax()
            self.select_player(selection)
            print(f'Selected {self.players.loc[selection]['name']}')


In [467]:
data = TeamAnalytics(mode='season', team_key='453.l.15482.t.3')
data.make_full_selections()

Selected Mikko Rantanen
Selected Sam Montembeault
Selected Cam Talbot
Selected Charlie Lindgren
Selected John Tavares
Selected Tage Thompson
Selected Matt Boldy
Selected Steven Stamkos
Selected Sam Bennett
Selected Mark Scheifele
Selected Zach Werenski
Selected Frank Vatrano
Selected Brandon Montour
Selected Thomas Chabot
Selected Seth Jones
Selected Shayne Gostisbehere
Selected Brad Marchand
Selected Nikita Zadorov


In [473]:
team_to_select = data.selected_team[:17]

In [472]:
for player in team_to_select:
    data = TeamAnalytics(mode='season', team_key='453.l.15482.t.3')
data.make_full_selections()

[np.int64(8478420),
 np.int64(8478470),
 np.int64(8475660),
 np.int64(8479292),
 np.int64(8475166),
 np.int64(8479420),
 np.int64(8481557),
 np.int64(8474564),
 np.int64(8477935),
 np.int64(8476460),
 np.int64(8478460),
 np.int64(8478366),
 np.int64(8477986),
 np.int64(8478469),
 np.int64(8477495),
 np.int64(8476906),
 np.int64(8473419),
 np.int64(8477507)]